In [1]:
import os
os.chdir("../")

In [2]:
from comchoice.voting import Voting
import pandas as pd
import numpy as np
from itertools import combinations
from comchoice.datasets import set_synthetic_election

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from comchoice.aggregate import borda, divisiveness, copeland, plurality, irv, black, k_approval, dowdall, kemeny_young, nanson, baldwin, minimax, schulze


In [4]:
help(copeland)

Help on function copeland in module comchoice.aggregate.copeland:

copeland(df, alternative='alternative', rank='rank', delimiter='>', show_rank=True, voter='voter', voters='voters')
    Copeland voting method (1951).
    
    Each voter ranks alternatives by preference. Next, we sort alternatives by the
    number of times they beat another alternative in a pairwise comparison.
    The top-1 on Copeland's method is considered a weak Condorcet winner.
    Likewise, if in an election of `n` alternatives, a alternative beats `n - 1`
    alternatives in pairwise comparison scenarios, it is also considered a Condorcet winner.
    
    Returns
    -------
    pandas.DataFrame:
        Election results using Copeland method.
    
    References
    ----------
    Copeland, A.H. (1951). A “reasonable” social welfare function, mimeographed. In: Seminar on applications of mathematics to the social sciences. Ann Arbor: Department of Mathematics, University of Michigan.



In [5]:
from comchoice.preprocessing import to_pairwise, to_individual_voter

In [6]:
data = []
data_dv = []
squares = 2
N = squares ** 2

import time


# Generates synthetic data for each square
# If we assume a geographical perspective, each square represents a region in a country
output = []
for square in range(1, N + 1):

    df = set_synthetic_election(
        alternatives=["A", "B", "C", "D"],
        n_alternatives=4,
        random_state=square,
        n_voters=1000
    )
    output.append(df)


# We calculate
for square in range(N):
    x = square // np.sqrt(N)
    y = square % np.sqrt(N)
    
    tmp = to_individual_voter(output[square])

    for method, name, is_divisiveness in [
        (borda(tmp), "Borda", False), 
        (divisiveness(tmp, convert_pairwise=True, method=borda), "Borda", True),
#         (copeland(tmp), "Copeland", False), 
#         (divisiveness(tmp, convert_pairwise=True, method=copeland), "Copeland", True),
        (dowdall(tmp), "Dowdall", False), 
        (divisiveness(tmp, convert_pairwise=True, method=dowdall), "Dowdall", True),
        (plurality(tmp), "Plurality", False), 
        (divisiveness(tmp, convert_pairwise=True, method=plurality), "Plurality", True),
        (minimax(tmp), "Minimax", False), 
        (divisiveness(tmp, convert_pairwise=True, method=minimax), "Minimax", True),
        (schulze(tmp), "Schulze", False), 
        (divisiveness(tmp, convert_pairwise=True, method=schulze), "Schulze", True),
#         (minimax(tmp, method="margins"), "Minimax (Margins)", False), 
#         (divisiveness(tmp, convert_pairwise=True, method=minimax), "Minimax (Margins)", True),
        (k_approval(tmp), "k-Approval (k=1)", False), 
        (divisiveness(tmp, convert_pairwise=True, method=k_approval), "k-Approval (k=1)", True)
#         (irv(tmp), "Hare"),
#         (black(tmp), "Black"),
#         (k_approval(tmp), "k-Approval"),
#         (dowdall(tmp), "Dowdall"),
#         (kemeny_young(tmp), "Kemeny-Young"),
#         (nanson(tmp), "Nanson"),
#         (baldwin(tmp), "Baldwin"),
    ]:

        start_time = time.time()

        rank = method.copy()
        rank["x"] = int(x)
        rank["y"] = int(y)
        rank["name"] = name
        rank["time"] = time.time() - start_time
        
        if is_divisiveness:
            data.append(rank)
        else:
            data_dv.append(rank)
    
data = pd.concat(data).reset_index(drop=True)
data

100%|██████████| 12/12 [00:00<00:00, 73.01it/s]


,alternative,value,rank,x,y,name,time
0,C,881.000000,1,0,0,Borda,35.940327
1,A,831.000000,2,0,0,Borda,35.940327
2,D,811.333333,3,0,0,Borda,35.940327
3,B,810.000000,4,0,0,Borda,35.940327
4,A,1.000000,1,0,0,Copeland,35.942885
...,...,...,...,...,...,...,...
107,D,1.000000,1,1,1,Minimax (Margins),30.203182
108,C,265.000000,1,1,1,k-Approval (k=1),30.205373
109,D,255.000000,2,1,1,k-Approval (k=1),30.205373
110,B,247.000000,3,1,1,k-Approval (k=1),30.205373


In [19]:
pd.merge(data_dv[data_dv["rank"] == 1], data[data["rank"] == 1], on=["x", "y", "name"])\
.sort_values("name")

,alternative_x,value_x,rank_x,x,y,name,time_x,alternative_y,value_y,rank_y,time_y
0,C,1521.000000,1.0,0,0,Borda,35.938997,C,881.000000,1,35.940327
30,D,1601.000000,1.0,1,0,Borda,32.899396,D,887.666667,1,32.900394
41,D,1560.000000,1.0,1,1,Borda,30.189063,C,883.000000,1,30.190080
11,D,1523.000000,1.0,0,1,Borda,33.870990,D,873.666667,1,33.872392
32,D,1.000000,1.0,1,0,Copeland,32.901459,B,1.000000,1,32.902523
31,D,1.000000,1.0,1,0,Copeland,32.901459,A,1.000000,1,32.902523
42,D,1.000000,1.0,1,1,Copeland,30.191131,A,1.000000,1,30.192307
43,D,1.000000,1.0,1,1,Copeland,30.191131,B,1.000000,1,30.192307
44,D,1.000000,1.0,1,1,Copeland,30.191131,C,1.000000,1,30.192307
19,D,0.833333,1.0,0,1,Copeland,33.873907,D,1.000000,1,33.875208


In [12]:
# data_dv = pd.concat(data_dv).reset_index(drop=True)
data_dv

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [8]:
dd = data.fillna(0).groupby(["name", "alternative"]).agg({"value": "count"})#.reset_index()
# dd.groupby(level=[0, 1]).max()
# dd.sort_values(["name", "value"], ascending=[True, False])
dd

value
name              alternative       
Borda             A              4
                  B              4
                  C              4
                  D              4
Copeland          A              4
                  B              4
                  C              4
                  D              4
Dowdall           A              4
                  B              4
                  C              4
                  D              4
Minimax           A              4
                  B              4
                  C              4
                  D              4
Minimax (Margins) A              4
                  B              4
                  C              4
                  D              4
Schulze           A              4
                  B              4
                  C              4
                  D              4
k-Approval (k=1)  A              4
                  B              4
                  C              4
                  D              4

In [9]:
data

,alternative,value,rank,x,y,name,time
0,C,881.000000,1,0,0,Borda,35.940327
1,A,831.000000,2,0,0,Borda,35.940327
2,D,811.333333,3,0,0,Borda,35.940327
3,B,810.000000,4,0,0,Borda,35.940327
4,A,1.000000,1,0,0,Copeland,35.942885
...,...,...,...,...,...,...,...
107,D,1.000000,1,1,1,Minimax (Margins),30.203182
108,C,265.000000,1,1,1,k-Approval (k=1),30.205373
109,D,255.000000,2,1,1,k-Approval (k=1),30.205373
110,B,247.000000,3,1,1,k-Approval (k=1),30.205373


In [10]:
df_sort = []
for i, tmp in enumerate(data[data["name"] == "Borda"].sort_values("alternative", ascending=False).iterrows()):
    index, df_tmp = tmp
    y_new = i % squares
    x_new = i // squares
    
    df_tmp.y
    df_sort.append({
        "x": df_tmp.x,
        "y": df_tmp.y,
        "x*": x_new,
        "y*": y_new
    })
    
df_sort = pd.DataFrame(df_sort)

In [11]:
data = pd.merge(data, df_sort, on=["x", "y"])

KeyError: 'x'

In [ ]:
time = data.groupby("name").agg({"time": "mean"}).to_dict()["time"]
time

In [ ]:
alternative_ids = {"A": 1, "B": 2, "C": 3, "D": 4}

sns.set(font_scale=1.5)
fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(12, 10))
# palette = ["#171F76", "#958BB6", "#00B493", "#006DBD"]
# palette = ["#0056FF", "#CECECE", "#CECECE", "#CECECE"]
palette = ["#00B6D7", "#006DBD", "#958BB6", "#004C97"]
# palette = ["#CECECE", "#CECECE", "#CECECE", "#004C97"]

for i, item in enumerate(data.groupby("name")):
    name, df_tmp = item
    
    im = df_tmp.pivot(index="x*", columns="y*", values="alternative").replace(alternative_ids)
    im = im.reindex(list(range(squares)), axis=0)
    im = im.reindex(list(range(squares)), axis=1)

    ax = sns.heatmap(im, cbar=False, cmap=palette, ax=axs[i // 4, i % 4])
    ax.tick_params(left=False, bottom=False) 

    for i in range(im.shape[0] + 1):
        ax.axhline(i, color="white", lw=1)
    for i in range(im.shape[1] + 1):
        ax.axvline(i, color="white", lw=1)

    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_title(f"{name} ({round(time[name], 2)}s)")
    
fig.tight_layout()